In [1]:
# data_path = 'E:/GitHub/CS-5302-Project-Group-15/Datasets/'

import os 
# PATH = 'ADD PATH HERE'
# os.chdir(PATH)
from IPython.display import Markdown
from python_scripts import llm_rag
from python_scripts import machine_translation
from python_scripts import text_to_speech
from python_scripts import whisper_setup
from python_scripts import llm_rag
from python_scripts import get_audio
from python_scripts import utils

PATH = os.getcwd().replace('\\', '/')
root_path = PATH + '/Datasets/MeDAL' # maybe caps 'data_path' too
data_path = root_path + '/training_data(2).jsonl'


# import replicate

# deployment = replicate.deployments.get("ubaidtariq8/llama2-med-genai")
# prediction = deployment.predictions.create(
#   input={"prompt": "..."}
# )
# prediction.wait()
# print(prediction.output)

In [2]:
# Not an exhaustive list
models = { 
    # Our custom model fine-tuned for medical diagnoses
    'llama_ours' : 'ubaidtariq8/llama2-med-genai',
    # Pre-trained 13 billion parameter LLM model
    'llama_13b': 'a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5',
    # Mixtral model from Mistral
    'mixtral': 'mistralai/mixtral-8x7b-instruct-v0.1',
    # Pre-trained 70 billion parameter LLM model from Meta
    'llama_70b': 'meta/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf'
}

In [ ]:
# Initialize the pipeline with our custom model and the root path
model = llm_rag.DocumentEmbeddingPipeline(
    model_version = models['llama_ours'], chroma_path = root_path # change this to experiment with different models available on Replicate
)

# Set up the environment for the model
model.setup_environment()

# The documents are located at the specified data path and are part of the "medal_filtered" collection
# The 'joining' parameter is set to False, so documents will not be joined together
# The 'persistent' parameter is set to True, so documents will be stored persistently
model.prepare_documents(data_path = data_path, collection_name = "medal_filtered",  joining = False, persistent = True)

# Embed the documents and index them for the model
model.embed_and_index()